In [164]:
from __future__ import print_function
import torch  #importing the torch library
import torch.nn as nn     #importing the neural network module
import torch.nn.functional as F      # importing 
import torch.optim as optim
from torchvision import datasets, transforms

In [165]:
class Net(nn.Module):         #base class for all the nn modules
    def __init__(self):       #initializing these parameters when the class object is created.
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,5,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.25)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(stride=2,kernel_size=2),
            nn.Conv2d(32,64,kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.25)
            
        )

        self.fc1 = nn.Sequential(
            nn.AvgPool2d(5)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(64,55),
            nn.ReLU(),
            nn.BatchNorm1d(55),
            nn.Dropout(0.25),
            # nn.Linear(32,16),
            # nn.ReLU(),
            # nn.Dropout(0.25),
            nn.Linear(55,10)
        )


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(x)
        x = x.view(-1,64)
        x = self.fc2(x)
        return F.log_softmax(x)

In [166]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             416
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         MaxPool2d-4           [-1, 16, 13, 13]               0
           Dropout-5           [-1, 16, 13, 13]               0
            Conv2d-6           [-1, 32, 11, 11]          12,832
              ReLU-7           [-1, 32, 11, 11]               0
       BatchNorm2d-8           [-1, 32, 11, 11]              64
         MaxPool2d-9             [-1, 32, 5, 5]               0
           Conv2d-10             [-1, 64, 5, 5]           2,112
             ReLU-11             [-1, 64, 5, 5]               0
      BatchNorm2d-12             [-1, 64, 5, 5]             128
          Dropout-13             [-1, 64, 5, 5]               0
        AvgPool2d-14             [-1, 6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [167]:

torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [168]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [169]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.034, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0591, Accuracy: 9807/10000 (98.07%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0497, Accuracy: 9829/10000 (98.29%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0281, Accuracy: 9914/10000 (99.14%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0256, Accuracy: 9916/10000 (99.16%)



epoch=19 loss=0.08013805001974106 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.29it/s]



Test set: Average loss: 0.0257, Accuracy: 9919/10000 (99.19%)



In [170]:
# class Net(nn.Module):         #base class for all the nn modules
#     def __init__(self):       #initializing these parameters when the class object is created.
#         super(Net, self).__init__()
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(1,16,5,padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(16),
#             nn.MaxPool2d(kernel_size=2,stride=2),
#             nn.Dropout(0.25)
#         )

#         self.conv2 = nn.Sequential(
#             nn.Conv2d(16,32,5,padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(stride=2,kernel_size=2),
#             nn.Conv2d(32,64,kernel_size=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.Dropout(0.25)
            
#         )

#         self.fc1 = nn.Sequential(
#             nn.AvgPool2d(5)
#         )

#         self.fc2 = nn.Sequential(
#             nn.Linear(64,55),
#             nn.ReLU(),
#             nn.BatchNorm1d(55),
#             nn.Dropout(0.25),
#             # nn.Linear(32,16),
#             # nn.ReLU(),
#             # nn.Dropout(0.25),
#             nn.Linear(55,10)
#         )


#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.fc1(x)
#         x = x.view(-1,64)
#         x = self.fc2(x)
#         return F.log_softmax(x)